<h1 style="font-size:3rem;color:orange;">WordleBot v.0.1 - Feedback-based guess space reduction</h1>

#### > Log Dec 26, 10.30 p.m. : In this version, we approach the problem in the most naive fashion, using feedback to filter a guess space repeatedly. We'll use each color separately, then the pairs of color, and all three at the same time. In testing performance, since the algorithm picks a guess from the guess space randomly, we select a random sample of a particular size from 2300 words as answers. This means that performance might differ in different test runs.

## Preliminaries

In [ ]:
import matplotlib.pyplot as plt
from math import log
import time

## Mathematics

In [ ]:
def convert_ternary(t):
    """


    Parameters
    ----------
    t : list
        Contains 05 elements, which can be 0, 1, or 2, denoting a feedback pattern.

    Returns
    -------
    int
        Base 10 representation of pattern.

    """
    return sum([t[i]*3**(4-i) for i in range(5)])

In [ ]:
def generate_ternary(n):
    """


    Parameters
    ----------
    n : int
        An integer, indicating length of number.

    Returns
    -------
    res : list
        List of all ternary numbers of length n.

    """
    res = list()
    s = [0] * n
    while True:
        temp = s[:]
        res.append(temp)
        i = n-1
        while i >= 0:
            if s[i] == 2:
                i -= 1
            else:
                break
        if i == -1:
            break
        else:
            temp = s[i]
            s[i] = temp + 1
            for j in range(i+1,n):
                s[j] = 0
    return res

## Wordle mechanics

In [ ]:
def get_feedback(guess,answer):
    """
    

    Parameters
    ----------
    guess : str
        Five-letter guess.
    answer : str
        Five-letter correct answer.

    Returns
    -------
    feedback : list
        Contains 05 elements, which can be 0, 1, or 2, denoting a feedback pattern.

    """
    #convert string to list
    temp = list(answer)
    answer = temp
    temp = list(guess)
    guess = temp
    
    #initialize
    feedback = ['']*5
    
    #isolate correctly placed letters
    for i in range(5):
        if guess[i] == answer[i]:
            feedback[i] = 2
            answer[i] = ''
            guess[i] = ''
    
    #isolate wrongly placed letters
    for i in range(5):
        if guess[i] == '': continue
        elif guess[i] in answer:
            feedback[i] = 1
            answer[answer.index(guess[i])] = ''
            guess[i] = ''
        else:
            feedback[i] = 0
    
    return feedback

## WordleBot mechanics

In [ ]:
def reduce_allowed_words(still_valid_words,guess,real_feedback,mode={0,1,2}):
    """
    

    Parameters
    ----------
    still_valid_words : list
        Contains words still possible as an answer.
    guess : str
        Five-letter guess.
    real_feedback : list
        Contains 05 elements, which can be 0, 1, or 2, denoting a feedback pattern.
    mode: set
        One of the seven filters: {0}, {1}, {2}, {0,1}, {0,2}, {1,2}, {0,1,2}

    Returns
    -------
    updated_allowed_words : list
        Updates allowed_words by retaining only words fitting the actual feedback.

    """
    n = 0
    accepted_feedback = real_feedback[:]
    
    #letter_feedback works like a mould, designated colors are retained, other positions can be changed arbitrary
    for i, letter_feedback in enumerate(real_feedback):
        if letter_feedback not in mode:
            n += 1
            accepted_feedback[i] = None
    
    accepted_feedbacks_enumerated = list()
    
    #generate_ternary(n) lists all combinations available to fill in the arbitrary positions
    for insertion in generate_ternary(n):
        temp = accepted_feedback[:]
        for i, letter_feedback in enumerate(temp):
            if letter_feedback == None:
                temp[i] = insertion.pop(0) #traverse left to right, popping a combination to fill in the mould
        accepted_feedbacks_enumerated.append(convert_ternary(temp))
    
    updated_allowed_words = list()
    for word in still_valid_words:
        feedback_enumerated = convert_ternary(get_feedback(guess,word))
        if feedback_enumerated in accepted_feedbacks_enumerated:
            updated_allowed_words.append(word)
    
    return updated_allowed_words

## Interactive interface

In [ ]:
def check_win(feedback):
    """
    

    Parameters
    ----------
    feedback : list
        Contains 05 elements, which can be 0, 1, or 2.

    Returns
    -------
    win : bool
        Becomes True when feedback is a list of 05 2's.

    """
    win = True
    for i in range(5):
        if feedback[i] != 2: 
            win = False
            break
    return win

In [ ]:
def wordlebot_interface(allowed_words,mode={0,1,2}):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains ~13000 allowed guesses.
    mode: set
        One of the seven filters: {0}, {1}, {2}, {0,1}, {0,2}, {1,2}, {0,1,2}
        

    Returns
    -------
    None.
    Prints the interactive program for user to play Wordle and input real feedback.

    """
    import random
    win = False
    still_valid_words = allowed_words
    guess_count = 0
    attempt_number = 0
    
    while not win:

        print("Guess #" + str(attempt_number+1))
        
        win = len(still_valid_words) == 1
        
        print("There are",len(still_valid_words),"left in the guess space.")
        
        if len(still_valid_words) > 10:
            print("By picking randomly, these are some of the words in the guess space:")
            for i in range(10):
                print(random.choice(still_valid_words))
        else:
            print("These are the words left in the guess space:")
            for i in range(len(still_valid_words)):
                print(still_valid_words[i])
        
        print()
        guess = input('> Enter your guess: ')
        real_feedback = list(map(int,input('>> Enter the feedback: ').split(' ')))
        
        if check_win(real_feedback) == True:
            print(">>> Complete!")
            break
            
        print()
        temp = reduce_allowed_words(still_valid_words,guess,real_feedback,mode)
        still_valid_words = temp

        attempt_number += 1
    
    return guess_count

## Performance testing

In [ ]:
def wordlebot_play(allowed_words,answer,mode={0,1,2}):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains ~13000 allowed guesses.
    answer : str
        Five-letter actual answer.
    mode: set
        One of the seven filters: {0}, {1}, {2}, {0,1}, {0,2}, {1,2}, {0,1,2}
        

    Returns
    -------
    guess_count : int
        Number of guesses needed to reach the actual answer.

    """
    import random
    still_valid_words = allowed_words
    guess_count = 0
    
    while True:
        
        win = len(still_valid_words) == 1
        
        guess = random.choice(still_valid_words)
        
        guess_count += 1
        
        real_feedback = get_feedback(guess,answer)
        
        if check_win(real_feedback):
            break
            
        temp = reduce_allowed_words(still_valid_words,guess,real_feedback,mode)
        still_valid_words = temp
    
    return guess_count

In [ ]:
def test_for_performance(allowed_words,possible_answers,sample_size,mode={0,1,2}):
    """
    

    Parameters
    ----------
    possible_answers : list
        Contains ~2300 human-curated possible answers.

    Returns
    -------
    None.
    Prints bar plot showing frequency of number of guesses needed.

    """
    import random
    #initialize
    performance_count = dict()
    
    #gameplay for ~2300 words in POSSIBLE_ANSWERS, with given sample size
    count = 1
    for i in range(sample_size):
        answer = random.choice(possible_answers)
        guess_count = wordlebot_play(allowed_words,answer,mode)

        performance_count[guess_count] = performance_count.get(guess_count,0) + 1
        print("Word " + str(count) + "/" + str(sample_size) + ": " + answer + " - Guesses taken: " + str(guess_count))
        count += 1
    
    #visualize
    x = list(range(1,max(performance_count.keys())+1))
    y = [performance_count.get(i,0) for i in x]
    plt.bar(x,y,color='royalblue',alpha=0.7)
    plt.grid(color='#95a5a6', linestyle='--', linewidth=1, axis='y', alpha=0.7)
    plt.title('WordleBot - Test performance - Mode ' + str(sorted(mode)))
    plt.xlabel('Number of guesses needed')
    plt.ylabel('Frequency')
    plt.show()
        
    #average number of guesses needed
    sum = 0
    win_count = 0
    for (guess_count,frequency) in performance_count.items():
        if guess_count <= 6:
            win_count += frequency
        sum += guess_count * frequency
    average = sum/sample_size
    win_rate = win_count/sample_size
    print("> Average number of guesses needed:",average)
    print("> Win rate:",win_rate)    

    #detailed distribution
    print("> Detailed distribution of number of guesses needed:")
    for guess_count in sorted(performance_count):
        print("- " + str(guess_count) + " guess(es): " + str(performance_count[guess_count]))

In [ ]:
def main():
    
    f = open('allowed_words.txt','r')
    ALLOWED_WORDS = list()
    for line in f:
        line = line.rstrip()
        ALLOWED_WORDS.append(line)
    f.close()

    f = open('possible_answers.txt','r')
    POSSIBLE_ANSWERS = list()
    for line in f:
        line = line.rstrip()
        POSSIBLE_ANSWERS.append(line)
    f.close()

    wordlebot_interface(ALLOWED_WORDS)

In [ ]:
if __name__ == "__main__":
    main()